In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import joblib

In [2]:
final_rmse = 5000

In [3]:
def download_data(symbol, start_date, end_date):
    df = yf.download(symbol, start=start_date, end=end_date)
    return df

In [4]:
def prepare_data(df, sequence_length):
    data = df.filter(['Close'])
    df = df.dropna()
    df = df[~df.index.duplicated(keep='last')]
    df = df.values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    X, y = [], []

    for i in range(sequence_length, len(scaled_data)):
        X.append(scaled_data[i-sequence_length:i, 0])
        y.append(scaled_data[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    split_index = int(len(scaled_data) * 0.8)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    
    return X_train, X_test, y_train, y_test, scaler

In [5]:
def build_gru_model(X_train, units=50, layers=2, activation='tanh', optimizer='adam'):
    model = Sequential()
    model.add(GRU(units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for _ in range(layers - 1):
        model.add(GRU(units, return_sequences=True))
    model.add(GRU(units*2))
    model.add(Dense(25, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [6]:
def train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers):
    
    global final_rmse
    
    loss_history = keras.callbacks.History()
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(f'model_{epochs}_{batch_size}_{sequence_length}_{units}_{layers}.h5', monitor='val_loss', save_best_only=True)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping, model_checkpoint, loss_history])

    gru_loss_history = loss_history.history['loss']
    
    gru_predictions = model.predict(X_test)
    gru_predictions = scaler.inverse_transform(gru_predictions)
    
    y_test = y_test.reshape(-1,1)
    y_test = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, gru_predictions))
    print(f"Root Mean Squared Error (Testing Dataset): {rmse}")
    
    if rmse <= final_rmse:
        final_rmse = rmse
        joblib.dump(model, "ETH_model_gru.pkl")
        plt.plot(y_test, label='Actual')
        plt.plot(gru_predictions, label='Predicted')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Time')
        plt.ylabel('Price')
        plt.legend()
        plt.savefig(f'ETH_actual_vs_predicted_GRU.png')
        plt.close()
    
        plt.figure(figsize=(10, 6))
        plt.plot(loss_history.history['loss'], label='Training Loss')
        plt.plot(loss_history.history['val_loss'], label='Validation Loss')
        plt.title('Epoch Loss Curve')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f'ETH_loss_curve_GRU.png')
        plt.close()
    
    return model, rmse


In [7]:
symbol = 'ETH-USD'
start_date = '2020-01-01'
end_date = '2024-01-31'

In [8]:
epochs_range = [100]
batch_sizes = [4,8]
sequence_lengths = [25,20]
units_range = [50, 100]
layers_range = [2, 3]

In [9]:
results = []

In [10]:
df = download_data(symbol, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [11]:
for epochs in epochs_range:
    for batch_size in batch_sizes:
        for sequence_length in sequence_lengths:
            for units in units_range:
                for layers in layers_range:
                    print(f"Training model with epochs={epochs}, batch_size={batch_size}, sequence_length={sequence_length}, units={units}, layers={layers}")
                    X_train, X_test, y_train, y_test, scaler = prepare_data(df, sequence_length)
                    model = build_gru_model(X_train, units=units, layers=layers)
                    model, rmse = train_model_and_evaluate_model(model, X_train, y_train, X_test, y_test, scaler, epochs, batch_size, sequence_length, units, layers)
                    results.append({'epochs': epochs, 'batch_size': batch_size, 'sequence_length': sequence_length, 'units': units, 'layers': layers, 'rmse': rmse})

Training model with epochs=100, batch_size=4, sequence_length=25, units=50, layers=2


Epoch 1/100

268/268 [==============================] - 21s 40ms/step - loss: 0.0037 - val_loss: 3.0381e-04
Epoch 2/100
  3/268 [..............................] - ETA: 7s - loss: 8.1418e-04

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 8s 31ms/step - loss: 0.0011 - val_loss: 1.5563e-04
Epoch 3/100
268/268 [==============================] - 8s 29ms/step - loss: 0.0011 - val_loss: 5.3086e-04
Epoch 4/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0011 - val_loss: 7.5514e-04
Epoch 5/100
268/268 [==============================] - 10s 36ms/step - loss: 0.0011 - val_loss: 1.6102e-04
Epoch 6/100
268/268 [==============================] - 10s 37ms/step - loss: 7.6842e-04 - val_loss: 2.2835e-04
Epoch 7/100
268/268 [==============================] - 10s 37ms/step - loss: 7.9419e-04 - val_loss: 1.0897e-04
Epoch 8/100
268/268 [==============================] - 8s 31ms/step - loss: 6.8173e-04 - val_loss: 1.2137e-04
Epoch 9/100
268/268 [==============================] - 9s 32ms/step - loss: 8.0702e-04 - val_loss: 2.5913e-04
Epoch 10/100
268/268 [==============================] - 8s 31ms/step - loss: 7.8037e-04 - val_loss: 1.2213e-04
Epoch 11/100
268/268 [===========

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


268/268 [==============================] - 11s 41ms/step - loss: 0.0015 - val_loss: 3.2715e-04
Epoch 3/100
268/268 [==============================] - 11s 41ms/step - loss: 0.0013 - val_loss: 1.5026e-04
Epoch 4/100
268/268 [==============================] - 12s 44ms/step - loss: 9.7752e-04 - val_loss: 3.4783e-04
Epoch 5/100
268/268 [==============================] - 12s 46ms/step - loss: 7.9519e-04 - val_loss: 1.2129e-04
Epoch 6/100
268/268 [==============================] - 11s 42ms/step - loss: 0.0012 - val_loss: 0.0014
Epoch 7/100
268/268 [==============================] - 11s 42ms/step - loss: 0.0011 - val_loss: 2.3572e-04
Epoch 8/100
268/268 [==============================] - 11s 40ms/step - loss: 9.2187e-04 - val_loss: 1.4980e-04
Epoch 9/100
268/268 [==============================] - 12s 46ms/step - loss: 8.7440e-04 - val_loss: 8.8111e-04
Epoch 10/100
268/268 [==============================] - 11s 42ms/step - loss: 6.8115e-04 - val_loss: 1.2133e-04
Epoch 11/100
268/268 [==========

...

134/134 [==============================] - 4s 30ms/step - loss: 0.0017 - val_loss: 6.9342e-04
Epoch 3/100
134/134 [==============================] - 4s 30ms/step - loss: 0.0015 - val_loss: 0.0015
Epoch 4/100
134/134 [==============================] - 4s 31ms/step - loss: 0.0011 - val_loss: 3.7314e-04
Epoch 5/100
134/134 [==============================] - 4s 30ms/step - loss: 8.7154e-04 - val_loss: 1.3990e-04
Epoch 6/100
134/134 [==============================] - 4s 30ms/step - loss: 7.9803e-04 - val_loss: 1.2981e-04
Epoch 7/100
134/134 [==============================] - 4s 30ms/step - loss: 7.8234e-04 - val_loss: 0.0019
Epoch 8/100
134/134 [==============================] - 4s 30ms/step - loss: 7.7507e-04 - val_loss: 4.0731e-04
Epoch 9/100
134/134 [==============================] - 4s 31ms/step - loss: 6.7491e-04 - val_loss: 3.6969e-04
Epoch 10/100
134/134 [==============================] - 4s 30ms/step - loss: 7.8373e-04 - val_loss: 2.9429e-04
Epoch 11/100
134/134 [===================

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 4s 29ms/step - loss: 0.0014 - val_loss: 4.0186e-04
Epoch 3/100
134/134 [==============================] - 4s 29ms/step - loss: 0.0011 - val_loss: 2.4670e-04
Epoch 4/100
134/134 [==============================] - 4s 29ms/step - loss: 7.8718e-04 - val_loss: 1.8148e-04
Epoch 5/100
134/134 [==============================] - 4s 28ms/step - loss: 8.1776e-04 - val_loss: 1.9607e-04
Epoch 6/100
134/134 [==============================] - 4s 30ms/step - loss: 9.1790e-04 - val_loss: 9.4602e-04
Epoch 7/100
134/134 [==============================] - 4s 28ms/step - loss: 7.1247e-04 - val_loss: 2.9218e-04
Epoch 8/100
134/134 [==============================] - 4s 29ms/step - loss: 5.6175e-04 - val_loss: 5.9302e-04
Epoch 9/100
134/134 [==============================] - 4s 30ms/step - loss: 8.2575e-04 - val_loss: 1.1114e-04
Epoch 10/100
134/134 [==============================] - 4s 31ms/step - loss: 8.9719e-04 - val_loss: 1.1558e-04
Epoch 11/100
134/134 [=======

C:\Users\Shreenav Dhakal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


134/134 [==============================] - 5s 37ms/step - loss: 0.0013 - val_loss: 5.6035e-04
Epoch 3/100
134/134 [==============================] - 7s 50ms/step - loss: 0.0014 - val_loss: 1.9568e-04
Epoch 4/100
134/134 [==============================] - 5s 35ms/step - loss: 0.0010 - val_loss: 9.5550e-04
Epoch 5/100
134/134 [==============================] - 5s 34ms/step - loss: 0.0011 - val_loss: 2.0656e-04
Epoch 6/100
134/134 [==============================] - 5s 35ms/step - loss: 9.6747e-04 - val_loss: 1.2361e-04
Epoch 7/100
134/134 [==============================] - 5s 34ms/step - loss: 7.0826e-04 - val_loss: 2.3036e-04
Epoch 8/100
134/134 [==============================] - 5s 34ms/step - loss: 0.0010 - val_loss: 1.3407e-04
Epoch 9/100
134/134 [==============================] - 5s 34ms/step - loss: 8.1736e-04 - val_loss: 3.5187e-04
Epoch 10/100
134/134 [==============================] - 5s 34ms/step - loss: 9.3751e-04 - val_loss: 0.0020
Epoch 11/100
134/134 [=======================

In [12]:
for result in results:
    print(result)

{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 50.19392084907332}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 50, 'layers': 3, 'rmse': 48.21089018708109}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 2, 'rmse': 47.671125557419856}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 25, 'units': 100, 'layers': 3, 'rmse': 48.83101165467385}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 2, 'rmse': 47.02926976548513}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 50, 'layers': 3, 'rmse': 47.259058428487094}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 2, 'rmse': 47.02101516576364}
{'epochs': 100, 'batch_size': 4, 'sequence_length': 20, 'units': 100, 'layers': 3, 'rmse': 47.32034734079032}
{'epochs': 100, 'batch_size': 8, 'sequence_length': 25, 'units': 50, 'layers': 2, 'rmse': 47.6878185495295}
{'epochs': 100